# Practice Text classification with Naive Bayes  
        
        
        
<h3>Abstract</h3>
<p>We will do text classification on a collection of Dutch parliamentary questions.
    The website <a href="https://zoek.officielebekendmakingen.nl/zoeken/parlementaire_documenten">officielebekendmakingen.nl</a>lets you search in "kamervragen".
    <!--You can donwload
    <a href='http://data.politicalmashup.nl/kamervragen/PoliDocs_Kamervragen.zip'>this zipfile with Kamervragen in XML</a>
    to see some of the  data in XML format. 
    It also contains style sheets to show the XML well in a browser.  
-->
    The <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/'>MYSQL directory</a> contains an <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR14807.xml'>example   Kamervraag XML file</a> and a file `kvr.csv.gz` with 40K kamervragen in a handy csv format. Note that in your browser you see the result of applying stylesheets. So choose View Source or open it in an editor.</p>

<h3>First exploration</h3>

See below.

<h2>Exercises</h2>

<p>We will use the fields in elements of the form <tt> &lt;item attribuut="Afkomstig_van"></tt> as our classes. 
    These are the ministeries to whom the question is addressed.
    An example is 
    <pre>
        &lt;item attribuut="Afkomstig_van">Landbouw, Natuurbeheer en Visserij (LNV)&lt;/item>
    </pre>
    Note that these labels are <strong>not normalized</strong>, see e.g. the counts below:
    <pre>
Justitie (JUS)                                                   3219
Volksgezondheid, Welzijn en Sport (VWS)                          2630
Buitenlandse Zaken (BUZA)                                        1796
Verkeer en Waterstaat (VW)                                       1441
Justitie                                                         1333
Sociale Zaken en Werkgelegenheid (SZW)                           1231
Onderwijs, Cultuur en Wetenschappen (OCW)                        1187
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer (VROM)     984
FinanciÃ«n (FIN)                                                   960
Volksgezondheid, Welzijn en Sport                                 951
Economische Zaken (EZ)                                            946
Buitenlandse Zaken                                                753
Binnenlandse Zaken en Koninkrijksrelaties (BZK)                   725
Verkeer en Waterstaat                                             724
Defensie (DEF)                                                    646
Sociale Zaken en Werkgelegenheid                                  607
Landbouw, Natuurbeheer en Visserij (LNV)                          586
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer            554
Onderwijs, Cultuur en Wetenschappen                               532
Vreemdelingenzaken en Integratie (VI)                             466
    </pre>
</p>


<h2>Form of handing in your final product</h2>

* An IPython notebook with for each question, a MarkDown cell containing the question, a code cell which solves the question, an output cell with the output, followed by a MarkDown cell with explanation/reflection  

In [ ]:
import pandas as pd

names=['jaar', 'partij','titel','vraag','antwoord','ministerie']

# Change to KVR1000.csv.gz if this becomes too slow for you
# kvrdf= pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR.csv.gz', 
kvrdf= pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR1000.csv.gz',
                   compression='gzip', sep='\t', 
                   index_col=0, names=names,
                   ) 

for kolom in names[1:]:
    kvrdf[kolom]= kvrdf[kolom].astype(str)
print(kvrdf.shape)
kvrdf.head()



In [ ]:
from nltk.corpus import stopwords
import nltk
from collections import Counter
import itertools
from math import log

DutchStop= stopwords.words('dutch')
allvragen= '\n'.join(list(kvrdf.titel))
classes = list(set(list(kvrdf.ministerie)))

Words are treated as lowercase & stopwords are filtered out below

In [ ]:
# Term frequencies per word

def strip_string(string):
    """
    
    :param string: string of unfiltered word/symbols
    :return: list of all tokens extracted from the string (lowercasing, stopwords, alpha)
    """
    return [w for w in nltk.word_tokenize(string.lower()) if w.isalpha() and not w in set(DutchStop)]
    
def str_to_tf(string):
    """
    
    :param string: string of unfiltered word/symbols
    :return: dictionary of all term frequencies: occurance of term in string
    """
    return Counter(strip_string(string))

In [ ]:
def str_to_tk(string):
    """ returns token count in a string
    """
    return len(strip_string(string))

## 1. 
Normalize the values for "ministerie" and choose 10 ministeries to work with.

In [ ]:
def determine_classes(classes):
    """ Remove any class that spans multiple ministeries
    
    :param classes: All classes that occur in kamervragen
    :return:
        norm_classes, a subset of classes with only the classes that span a single ministerie.
    """
    norm_classes = set()
    
    for c in classes:
        add = True

        if c == 'nan':
            continue

        for nc in norm_classes:
            if nc in c:
                add = False
                break
            elif c in nc:
                norm_classes.remove(nc)
                break

        if add:
            norm_classes.add(c)
            
    return norm_classes


def normalize_class(c):
    """ Normalize class c by replacing different representations of the e by a normal e
    
    :param c: str, a single class name
    :return:
        str, a single class name with 'normal' e's and no trailing whitespace.
    """
    nc = ""
    parenthesis = False
    
    for char in c:
        if char == '(':
            parenthesis = True
            
        elif char == 'ë':
            char = 'e'
        elif char == 'Ã':
            char = 'e'
        elif char == '«':
            char = ''
            
        if not parenthesis:
            nc += char
            
        if char == ')':
            parenthesis = False
    
    return nc.strip()


def choose_10_classes(class_rows, norm_classes):
    """ Only return the 10 most occuring classes.
    
    :param class_rows: dict, a dictionary mapping a class name to all rows that class occurs in.
    :param norm_classes: set, all normalized class names of classes that only span a single ministerie.
    
    :return:
        set, the normalized names of the 10 most occuring classes
    """
    count = Counter()
    for key, rows in class_rows.items():
        key = normalize_class(key)
        
        if key in norm_classes:
            count[key] += len(rows)
            
    return set([name for name, _ in count.most_common(10)])


def kvrdf_to_10_classes(kvrdf, norm_classes):
    """ Return a copy of kvrdf with normalized class names and only containing rows with classes in norm_classes
    
    :param kvrdf: pd.Dataframe, containing all downloaded information: questions, ministerie, answers, etc.
    :param norm_classes: set, containing all normalized class names of the 10 most occuring classes.
    
    :return:
        pd.Dataframe, kvrdf but with all rows removed that don't contain any of the ministeries in norm_classes.
    """   
    for c in set(kvrdf.ministerie):
        nc = normalize_class(c)
        
        if not (nc in norm_classes):
            kvrdf = kvrdf[kvrdf.ministerie != c]
            
        else:
            kvrdf.loc[kvrdf["ministerie"] == c, "ministerie"] = nc
            
    return kvrdf

# classes is a set of strings of all classes that are in kvrdf
classes = set(kvrdf.ministerie)

# norm classes is a set of strings of classes that only span a single ministerie with normalized names.
norm_classes = {normalize_class(c) for c in determine_classes(classes)}

# class_rows_full is a dictionary mapping a class name to all rows that contain that class.
class_rows_full = {c:kvrdf.loc[kvrdf.ministerie == c] for c in classes}

# classes is adjusted so it now is a set that contains the normalized strings of the 10 most occuring classes.
classes = choose_10_classes(class_rows_full, norm_classes)
print(classes)

# kvrdf is adjusted so any rows that don't contain any class in 'classes' are removed.
# also, all class names are normalized.
kvrdf = kvrdf_to_10_classes(kvrdf, classes)
kvrdf.head()

Classes are extracted from the ministerie column in the dataframe. Any classes that don't occur often or span multiple ministeries, are removed from the dataframe. All other classes have their names normalized.

The normalization is done by changing all variants of the 'e' in a class to a normal 'e'. Also any class that is named 'nan' were removed. And finally, any starting or trailing whitespace were removed from the class name.

## 2.
Implement the two algorithms in Fig MRS.13.2, using your earlier code for creating term and document frequencies. It might be easier to use the representation and formula given in MRS section 13.4.1.

In [6]:
#document frequencies per class per term
def TrainMultinomialNB(pd_df, classes):
    """ more reference from p258 onwards of MRS
    
    :param classes: classes which are taken into account for training
    :param pd_df: kamervragen dataframe
    :return:
        V: set, of terms which which are used for classification
        class_priors: dictionary, prior probabilities for each class: P(c) prior[class]
        cond_prob: dictionary, conditional probabilities for each term per class: P(t|c) cond[class][term]
        dfdict: dictionary with document frequencies
        classes: list, classes actually used, temporary until class input is synchronised with pandas dataframe
    """
    class_frequency = Counter([c for c in pd_df.ministerie])
    
    # P(c)
    class_priors = {c:class_frequency[c]/sum([class_frequency[cn] for cn in classes]) for c in classes}

    # document frequencies per term
    k = [list(set(strip_string(t))) for t in kvrdf.titel]
    print(kvrdf)
    dfdict = Counter(list(itertools.chain.from_iterable(k)))

    # vocabulary
    V = set(dfdict.keys())
    
    # complete rows per class
    class_rows_full = {c:kvrdf.loc[kvrdf.ministerie == c] for c in classes}

    # term frequency per class
    class_tf = {c:str_to_tf('\n'.join(list(class_rows_full.get(c).titel))) for c in classes}
    
    # token count per class (for sum over Tct' in (t' in V), (p259, formula 13.6)
    class_tk = {c:str_to_tk('\n'.join(list(class_rows_full.get(c).titel))) for c in classes}

    # P(t|c)
    cond_prob = {t:{c:(class_tf[c][t] + 1)/(class_tk[c] + 1) for c in classes} for t in V}
    return V, class_priors, cond_prob, dfdict, classes

def filter_vocab(W, V):
    """ 
    
    :param W: list; stripped query
    :param V: set; of vocab used for classification (determined by one or anoter method)
    :return: a list of words that occur in the used vocabulary
    """
    return [w for w in W if w in V]

def ApplyMultinomialNB(classes, priors, conditionals, d, Vocab=None):
    """ Applies a Naive Bayes classifier on previously derived priors & conditionals on a query d

    :param classes: list, optional ministeries to classify between 
    :param Vocab: set, of terms which which are used for classification
    :param priors: dictionary, prior probabilities for each class: P(c) prior[class]
    :param conditionals: dictionary, conditional probabilities for each term per class: P(t|c) cond[class][term]
    :param d: string, query
    :return: best classification for query
    """
    W = strip_string(d)
    if Vocab:
        W = filter_vocab(W, Vocab)
    score = {c:log(priors[c]) for c in classes}
    for k in score.keys():
        score[k] += sum([log(conditionals[t][k]) for t in W])    
    best_class = sorted(score, key = lambda key: score[key])[0]
    return best_class



## 3.
On this collection, train NB text classifiers for 10 different classes with enough and interesting data.

In [ ]:
V, class_priors, cond_prob, dfdict, classes = TrainMultinomialNB(kvrdf, classes)

ApplyMultinomialNB(classes, class_priors, cond_prob, "vragen")


## 4.
Compute for each term and each of your 10 classes its utility for that class using mutual information.

In [8]:
def extract_class_df(pddf, classes):
    class_dfdict = {c:{} for c in classes}

    for i, c in enumerate(classes):
        for d in kvrdf[kvrdf.ministerie == c].titel:
            for elem in set(strip_string(d)):
                if class_dfdict[c].get(elem, 0):
                    class_dfdict[c][elem] += 1
                else:
                    class_dfdict[c][elem] = 1
    return class_dfdict

cddf = extract_class_df(kvrdf, classes)

In [9]:
def MI(doc_freq, class_doc_freq):
    """ creates a mutual information dictionary for each term, class pair
    
    :param doc_freq: dictionary, contains documents frequencies per term, doc_freq[term]
    :param class_doc_freq: dictionary, contains documents frequencies per term per class, cdf[class][term]
    :return MIdict: dictionary, contains mutual information for each class/term pair, MI[class][term]
    """
    N = sum(doc_freq.values())
    MIdict = {}
    for c in class_doc_freq.keys():
        MIdict[c] = {}
        for t in class_doc_freq[c].keys():
            N11 = class_doc_freq[c][t]
            N10 = doc_freq[t] - N11
            N01 = sum(class_doc_freq[c].values()) - class_doc_freq[c][t]
            N00 = N - doc_freq[t] - (sum(class_doc_freq[c].values()) - class_doc_freq[c][t])
            N1 = N11 + N10
            N0 = N01 + N00
            if N11:
                MIdict[c][t] = N11/N * log( (N*N11)/(N1*N1), 2)
            if N01:
                MIdict[c][t] += N01/N * log( (N*N01)/(N0*N1) , 2)
            if N10:
                MIdict[c][t] += N10/N * log( (N*N10)/(N1*N0) , 2)
            if N00:
                MIdict[c][t] += N00/N * log(  (N*N00)/(N0*N0), 2)
    return MIdict

MIdict = MI(dfdict, cddf)


## 5.
For each class, show the top 10 words as in Figure 13.7 in MRS.

In [10]:
from IPython.display import display, Markdown, Latex



In [22]:
def top_MI(MIdict, N):
    """ 
    
    :param MIdict: a dictionary with mutual information for each MIdict[class][term]
    :param N: top N MI terms returned per class
    :return: a dictionary of the top N MI predictors of each class    
    """
    resdict = {}
    for c in MIdict.keys():
        resdict[c] = sorted(MIdict[c].items(), key = lambda key: -MIdict[c][key[0]])[:N]
    return resdict

def top_MI_to_markdown(topMI):
    firstline = classes_table_markdown(topMI)
    rest_of_table = values_table_markdown(topMI)
    return firstline + rest_of_table


def classes_table_markdown(topmidict):
    return "| []() |" + " | []() | ".join(topmidict.keys()) + "|\n" + "|" + " | ".join(['----- | -----' for _ in topmidict.keys()]) + "| \n"

def values_table_markdown(topmidict):
    returnstring = ""

    for i, elem in enumerate(topmidict[list(topmidict.keys())[0]]):
        returnstring += " | " + " | ".join([pair_to_markdown(topmidict[c][i]) for c in topmidict.keys()]) + " | \n"
    return returnstring
            
def pair_to_markdown(pair):
    return pair[0] + "|" + str(pair[1])


def topMI_to_vocab(topmidict):
    vocab_l_o_l = [[elem[0] for elem in topmidict[c]] for c in topmidict.keys()]
    return set(list(itertools.chain.from_iterable(vocab_l_o_l)))



In [28]:
topfeatures = top_MI(MIdict, 100)


# here we remove terms that occur only a few times, to give us back a more robust database
dfdict2 = {k:v for k,v in dfdict.items() if v > 5}
cddf2 = {c:{t:cddf[c][t] for t in cddf[c] if t in dfdict2.keys()} for c in cddf.keys()}
MIdict2 = MI(dfdict2, cddf2)
topfeatures2 = top_MI(MIdict2, 85)

topfeatures_MD2 = top_MI_to_markdown(topfeatures2)
display(Markdown(topfeatures_MD2))

| []() |Justitie | []() | Volksgezondheid, Welzijn en Sport | []() | Buitenlandse Zaken | []() | Verkeer en Waterstaat | []() | Sociale Zaken en Werkgelegenheid | []() | Onderwijs, Cultuur en Wetenschappen | []() | Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer | []() | Landbouw, Natuurbeheer en Visserij | []() | Economische Zaken | []() | Defensie|
|----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | -----| 
 | asielzoeker|1.2835610580560481 | vergoed|0.9923884480113581 | the|0.5897203603790898 | landen|0.49681499011124053 | bijstandswet|0.4900608266294052 | bijdrage|0.47301653654544257 | woningen|0.3740385374949737 | korte|0.18969236885847 | financieele|0.20253133819383656 | luchthaven|0.16870077080292553 | 
 | afkomstig|1.2817061473821174 | verzorgingshuizen|0.9923884480113581 | frankrijk|0.5890336225929294 | vervoer|0.4960856247957784 | europees|0.4891058728843098 | verplicht|0.47301653654544257 | vrij|0.3715420413721538 | groot|0.18969236885847 | toegang|0.20253133819383656 | vliegbasis|0.16870077080292553 | 
 | burgemeester|1.2817061473821174 | arts|0.9911626190931526 | voorstel|0.5890336225929294 | the|0.4955505464571678 | vervoer|0.48783374405740754 | gebied|0.47246981336615507 | inspectie|0.37074226455122417 | afkomstig|0.18893268112545103 | hierover|0.20126365166398277 | ruim|0.16735157852067656 | 
 | hoge|1.2812265039201665 | wachtlijsten|0.9903895913518133 | internationale|0.5890336225929294 | den|0.4955505464571678 | toegang|0.48783374405740754 | zoals|0.47246981336615507 | betaalt|0.37074226455122417 | risico|0.18839534206236608 | tijdens|0.20126365166398277 | personeel|0.16735157852067656 | 
 | hierover|1.2812265039201665 | vergoeding|0.989838063091725 | nigeriaanse|0.5890336225929294 | bepleit|0.4955505464571678 | wonende|0.48783374405740754 | kinderopvang|0.47246981336615507 | fraude|0.37013691844119806 | inspectie|0.18839534206236608 | dienst|0.20126365166398277 | persbericht|0.16735157852067656 | 
 | nigeriaanse|1.2812265039201665 | gebied|0.989838063091725 | voorgenomen|0.5885411008330961 | financieele|0.49523720121516757 | risico|0.48783374405740754 | wachtlijsten|0.47246981336615507 | overheid|0.37013691844119806 | mogelijkheden|0.18839534206236608 | vergunning|0.20126365166398277 | risico|0.16735157852067656 | 
 | belgië|1.2812265039201665 | personeel|0.989838063091725 | luchthaven|0.5885411008330961 | ruim|0.49523720121516757 | uitspraak|0.48783374405740754 | staatssecretaris|0.47246981336615507 | vergunning|0.37013691844119806 | tijdschrift|0.18839534206236608 | belgië|0.20126365166398277 | staatssecretaris|0.16735157852067656 | 
 | verstrekken|1.2812265039201665 | alle|0.989838063091725 | duitsland|0.5885411008330961 | mogelijkheden|0.49523720121516757 | tijdschrift|0.48783374405740754 | middel|0.47246981336615507 | internationale|0.37013691844119806 | fraude|0.18839534206236608 | persbericht|0.20126365166398277 | duitsland|0.16735157852067656 | 
 | uitzending|1.280940853683202 | dienst|0.989838063091725 | bepleit|0.5885411008330961 | groot|0.49523720121516757 | europa|0.48783374405740754 | internationale|0.47214484458286243 | korte|0.37013691844119806 | frankrijk|0.18839534206236608 | duitsland|0.20126365166398277 | voorstel|0.16735157852067656 | 
 | alleenstaande|1.280940853683202 | stichting|0.9894805866749404 | alleenstaande|0.5885411008330961 | europa|0.49523720121516757 | kinderopvang|0.48783374405740754 | tijdens|0.47214484458286243 | persbericht|0.37013691844119806 | duitsland|0.18839534206236608 | overheid|0.20126365166398277 | defensie|0.16230178432465694 | 
 | wonende|1.280940853683202 | middel|0.9894805866749404 | tijdens|0.5885411008330961 | korte|0.49523720121516757 | mogelijkheden|0.48783374405740754 | risico|0.47214484458286243 | plannen|0.37013691844119806 | plannen|0.18839534206236608 | europa|0.20126365166398277 | commerciële|0.15788196972798507 | 
 | vliegbasis|1.280940853683202 | inspectie|0.9894805866749404 | europa|0.5885411008330961 | overheid|0.49523720121516757 | verplicht|0.48751656662932596 | personeel|0.47214484458286243 | vraag|0.3697534485046589 | daarvan|0.18839534206236608 | vervoer|0.20126365166398277 | vn|0.15788196972798507 | 
 | vrij|1.280940853683202 | kinderopvang|0.9894805866749404 | hierover|0.5885411008330961 | hoge|0.49523720121516757 | daarvan|0.48751656662932596 | plannen|0.47214484458286243 | uitspraak|0.3697534485046589 | vraag|0.18839534206236608 | zoals|0.20126365166398277 | miljoen|0.15788196972798507 | 
 | vraag|1.280940853683202 | regeling|0.9894805866749404 | zoals|0.5885411008330961 | voorgenomen|0.49523720121516757 | regeling|0.48751656662932596 | stichting|0.47214484458286243 | frankrijk|0.3697534485046589 | resultaten|0.17804728640169099 | verplicht|0.20126365166398277 | kader|0.15788196972798507 | 
 | voorgenomen|1.280940853683202 | den|0.9894805866749404 | bijdrage|0.588270241490928 | plannen|0.49523720121516757 | steeds|0.48751656662932596 | vergunning|0.47214484458286243 | verplicht|0.3697534485046589 | commerciële|0.17804728640169099 | betalen|0.19085379241710965 | binnen|0.15788196972798507 | 
 | ruim|1.280940853683202 | steeds|0.9894805866749404 | ruim|0.588270241490928 | fraude|0.49523720121516757 | fraude|0.48751656662932596 | daarvan|0.47214484458286243 | alle|0.3697534485046589 | binnen|0.17804728640169099 | kamerstuk|0.19038853023825214 | voormalige|0.15788196972798507 | 
 | stichting|1.280940853683202 | vergunning|0.9893446089884042 | alle|0.588270241490928 | uitzending|0.49523720121516757 | staatssecretaris|0.48751656662932596 | den|0.47214484458286243 | bijstandswet|0.3697534485046589 | geven|0.17804728640169099 | antwoord|0.19038853023825214 | koninklijke|0.15260141727824428 | 
 | tijdens|1.280940853683202 | alleenstaande|0.9893446089884042 | burgemeester|0.588270241490928 | verstrekken|0.49523720121516757 | dienst|0.48751656662932596 | regeling|0.47214484458286243 | dienst|0.3697534485046589 | bestrijdingsmiddelen|0.17190102453857833 | regering|0.19038853023825214 | relatie|0.15164414882309712 | 
 | middel|1.280940853683202 | groot|0.9893446089884042 | uitzending|0.588270241490928 | betaalt|0.49523720121516757 | alleenstaande|0.48751656662932596 | alle|0.47214484458286243 | uitzending|0.3697534485046589 | gezien|0.1695542662094155 | resultaten|0.19038853023825214 | parool|0.14971775654291222 | 
 | gebied|1.2808766436912407 | plannen|0.9893446089884042 | wonende|0.588270241490928 | vergoeding|0.49523720121516757 | financieele|0.48751656662932596 | mogelijkheden|0.47214484458286243 | hoge|0.3697534485046589 | toelating|0.16298552105036343 | binnen|0.19038853023825214 | juni|0.14971775654291222 | 
 | staatssecretaris|1.2808766436912407 | vliegbasis|0.9893446089884042 | europees|0.588270241490928 | stichting|0.49523720121516757 | persbericht|0.48751656662932596 | belgië|0.47214484458286243 | daarvan|0.3697534485046589 | hiervan|0.16225730363415014 | enkele|0.19038853023825214 | herdruk|0.14971775654291222 | 
 | steeds|1.2808766436912407 | voorgenomen|0.9893446089884042 | steeds|0.588270241490928 | regeling|0.49523720121516757 | werknemers|0.4674634352100586 | uitzending|0.47214484458286243 | den|0.3697534485046589 | termijn|0.16225730363415014 | gehandicapte|0.19038853023825214 | vrouwen|0.14971775654291222 | 
 | groot|1.2808766436912407 | tijdschrift|0.9893446089884042 | betaalt|0.588270241490928 | besluitvorming|0.4767200314895609 | gehandicapte|0.46682711122895315 | steeds|0.47214484458286243 | risico|0.3697534485046589 | voornemen|0.16225730363415014 | relatie|0.18142430755068953 | eerder|0.14971775654291222 | 
 | vergunning|1.2808766436912407 | verstrekken|0.9893446089884042 | hoge|0.588270241490928 | enkele|0.4738135434378534 | antwoord|0.46610051546939657 | tijdschrift|0.47214484458286243 | burgemeester|0.3697534485046589 | gesteld|0.16167488144615935 | bedrijf|0.18142430755068953 | opdracht|0.1425504661100925 | 
 | bijdrage|1.2808766436912407 | daarvan|0.9893446089884042 | fraude|0.588270241490928 | kader|0.4738135434378534 | commerciële|0.46610051546939657 | vergoeding|0.47214484458286243 | antwoorden|0.35279187159471465 | dreigende|0.16128365610220596 | groningen|0.18142430755068953 | februari|0.1425504661100925 | 
 | zoals|1.2808766436912407 | bijdrage|0.9893446089884042 | verstrekken|0.588270241490928 | zeeland|0.4738135434378534 | invoering|0.46610051546939657 | vraag|0.47214484458286243 | binnen|0.35279187159471465 | af|0.16128365610220596 | artikelen|0.18142430755068953 | af|0.1425504661100925 | 
 | landen|1.2808766436912407 | wonende|0.9893446089884042 | overheid|0.588270241490928 | gehandicapte|0.4735576365733657 | binnen|0.46610051546939657 | uitspraak|0.47214484458286243 | mei|0.3524658386941456 | februari|0.16128365610220596 | betrekking|0.18142430755068953 | marechaussee|0.1395970220220597 | 
 | luchthaven|1.2808766436912407 | belgië|0.9893446089884042 | landen|0.588270241490928 | invoering|0.4735576365733657 | resultaten|0.46610051546939657 | voortgezet|0.45594071110992385 | miljoen|0.3524658386941456 | commissie|0.15430230623450875 | regio|0.18100681847698633 | blijkt|0.1361695092118014 | 
 | regeling|1.2808766436912407 | bepleit|0.9893446089884042 | daarvan|0.588270241490928 | antwoord|0.4735576365733657 | zeeland|0.46610051546939657 | studenten|0.4544765495967532 | kamerstuk|0.3524658386941456 | amerikaanse|0.15430230623450875 | regionale|0.18100681847698633 | twee|0.1361695092118014 | 
 | europees|1.2808766436912407 | voorstel|0.9893446089884042 | vn|0.5664217227890039 | betalen|0.4735576365733657 | kamerstuk|0.46610051546939657 | nijmegen|0.45152335251405096 | komen|0.3382835601981293 | rond|0.15430230623450875 | komen|0.18100681847698633 | amerikaanse|0.1361695092118014 | 
 | frankrijk|1.2808766436912407 | ruim|0.9893446089884042 | regering|0.5654554406315266 | geven|0.4735576365733657 | moeten|0.4482093608156502 | gehandicapte|0.45152335251405096 | betrekking|0.3382835601981293 | gelderlander|0.15430230623450875 | herdruk|0.18100681847698633 | kosovo|0.13130628102998976 | 
 | bepleit|1.2808766436912407 | persbericht|0.9893446089884042 | mei|0.5641063516678388 | aanleg|0.4566960182953559 | regio|0.4482093608156502 | geven|0.45152335251405096 | aanleg|0.337807710687141 | aanzien|0.15430230623450875 | kamer|0.18100681847698633 | schiphol|0.13130628102998976 | 
 | duitsland|1.2808766436912407 | uitspraak|0.9893446089884042 | antwoord|0.5636858918003829 | cobouw|0.4566960182953559 | kamer|0.4477999424841752 | kader|0.451255821862467 | groningen|0.337807710687141 | grond|0.14884379376519163 | aanleg|0.18100681847698633 | waaruit|0.13042469916167465 | 
 | tijdschrift|1.2808766436912407 | overheid|0.9893446089884042 | miljoen|0.5636858918003829 | extra|0.455990966389489 | mensen|0.4477999424841752 | kamerstuk|0.451255821862467 | cobouw|0.337807710687141 | uitspraken|0.1422990129810944 | gezien|0.18100681847698633 | courant|0.13042469916167465 | 
 | arts|1.2808766436912407 | toegang|0.9893446089884042 | studenten|0.5634724718245137 | metro|0.4554305494571778 | particuliere|0.4477999424841752 | enkele|0.451255821862467 | herdruk|0.3375294527508873 | aanvullende|0.13733349394286276 | eerder|0.18100681847698633 | kosten|0.1252051539903724 | 
 | mogelijkheden|1.2808766436912407 | financieele|0.9893446089884042 | voormalige|0.5634724718245137 | openbaar|0.4550249704001076 | betrekking|0.4477999424841752 | resultaten|0.451255821862467 | bedrijf|0.3375294527508873 | grote|0.1370644265647934 | openbaar|0.18100681847698633 | informatie|0.1252051539903724 | 
 | vergoeding|1.2808766436912407 | vraag|0.9893446089884042 | kamerstuk|0.5634724718245137 | mensen|0.4550249704001076 | eerder|0.4477999424841752 | mei|0.451255821862467 | artikelen|0.3375294527508873 | gemeenten|0.1370644265647934 | beleid|0.17276562864421574 | politie|0.12042667497133147 | 
 | voorstel|1.2808766436912407 | staatssecretaris|0.9893446089884042 | betalen|0.5634724718245137 | regionale|0.4550249704001076 | bedrijf|0.4477999424841752 | commerciële|0.451255821862467 | zaken|0.3375294527508873 | college|0.13344725675849609 | heer|0.17276562864421574 | via|0.11602381708401485 | 
 | personeel|1.2808766436912407 | zorg|0.95553398636567 | antwoorden|0.5634724718245137 | gezien|0.45481683984205945 | vanuit|0.4477999424841752 | invoering|0.451255821862467 | relatie|0.3375294527508873 | algemene|0.13223928320015105 | opdracht|0.17276562864421574 | waardoor|0.11602381708401485 | 
 | betaalt|1.2808766436912407 | zeeland|0.9528055088449009 | radio|0.5634724718245137 | juni|0.45481683984205945 | financiering|0.4477999424841752 | betalen|0.451255821862467 | mensen|0.3375294527508873 | via|0.13223928320015105 | terwijl|0.17276562864421574 | terzake|0.11194470244100901 | 
 | uitspraak|1.2808766436912407 | werknemers|0.9523508733692614 | resultaten|0.5634724718245137 | vanuit|0.45481683984205945 | gezien|0.4475879795757095 | voormalige|0.451255821862467 | gesteld|0.32462611024274357 | besluit|0.12799026873026442 | dreigende|0.17276562864421574 | bedrijven|0.11194470244100901 | 
 | illegalen|1.2369137846566651 | radio|0.9523508733692614 | herdruk|0.5425592895569742 | vrouwen|0.45481683984205945 | parool|0.4475879795757095 | financiering|0.4338367881007579 | termijn|0.32462611024274357 | terzake|0.12776703392278582 | commissie|0.16695736257698485 | besluit|0.11194470244100901 | 
 | voormalige|1.2356031376172936 | geven|0.9523508733692614 | juni|0.5420306263756112 | parool|0.45481683984205945 | vanwege|0.4475879795757095 | groningen|0.43341956371151047 | opdracht|0.32462611024274357 | mogelijk|0.12360209346340861 | blijkt|0.16576485728465973 | leden|0.10837634696957138 | 
 | antwoorden|1.2356031376172936 | mei|0.9520654631642567 | regionale|0.5420306263756112 | kamer|0.45481683984205945 | herdruk|0.4475879795757095 | regionale|0.43341956371151047 | vergaderjaar|0.3243887600462033 | leden|0.12360209346340861 | amerikaanse|0.16542404535354543 | verband|0.10627794727084308 | 
 | miljoen|1.2352203902189411 | betalen|0.9520654631642567 | kamer|0.5420306263756112 | financiering|0.45481683984205945 | komen|0.4475879795757095 | juni|0.4331998091204545 | auto|0.3243887600462033 | gevolg|0.12360209346340861 | twee|0.16542404535354543 | gebruik|0.10459803934737805 | 
 | radio|1.2352203902189411 | invoering|0.9520654631642567 | financiering|0.5420306263756112 | bedrijf|0.45481683984205945 | zaken|0.4475879795757095 | kamer|0.4331998091204545 | eigen|0.3243887600462033 | gebruik|0.12024400788688394 | bestuur|0.16542404535354543 | problemen|0.10459803934737805 | 
 | enkele|1.2352203902189411 | nijmegen|0.9520654631642567 | gezien|0.5420306263756112 | stellen|0.45481683984205945 | werken|0.4318256556233676 | gezien|0.4331998091204545 | af|0.3243887600462033 | maatregelen|0.10935777227267277 | scholen|0.16542404535354543 | gaan|0.10182000523217702 | 
 | nijmegen|1.2352203902189411 | resultaten|0.9520654631642567 | stellen|0.5420306263756112 | auto|0.4399814776736508 | zodat|0.4318256556233676 | vanwege|0.4331998091204545 | blijkt|0.3132532259291239 | handelsblad|0.10627935416787543 | laten|0.16542404535354543 | kinderen|0.10126795340074476 | 
 | kader|1.235006809474505 | commerciële|0.9519869291654123 | betrekking|0.5420306263756112 | vergaderjaar|0.43884550811701306 | eigen|0.43146401351614605 | stellen|0.4331998091204545 | staat|0.3132532259291239 | gevolgen|0.10458164983530321 | schiphol|0.1588102514968136 | waarbij|0.09830857617905969 | 
 | antwoord|1.235006809474505 | binnen|0.9519869291654123 | artikelen|0.5420306263756112 | terwijl|0.43884550811701306 | heer|0.43146401351614605 | komen|0.4331998091204545 | krijgen|0.3128660591221645 | nieuwe|0.10411111180804296 | waaruit|0.1588102514968136 | ministerie|0.09517466343311284 | 
 | commerciële|1.235006809474505 | kader|0.9519869291654123 | mensen|0.5420306263756112 | af|0.43884550811701306 | beleid|0.43146401351614605 | regio|0.4331998091204545 | drie|0.3126644820491812 | jaar|0.10372967246367429 | uitspraken|0.15279739400122094 | maatregelen|0.09517466343311284 | 
 | invoering|1.235006809474505 | defensie|0.9519869291654123 | vanuit|0.5420306263756112 | voornemen|0.4384877053663824 | geld|0.43146401351614605 | programma|0.4331998091204545 | nova|0.3126644820491812 | inzake|0.10346131894667421 | kosten|0.15279739400122094 | handelsblad|0.09282789746782741 | 
 | regering|1.2350000464525832 | kamerstuk|0.9519869291654123 | beleid|0.5246424456566791 | heer|0.4384877053663824 | gesteld|0.43146401351614605 | extra|0.4331998091204545 | gelderlander|0.3126644820491812 | minister|0.10346131894667421 | medewerkers|0.15279739400122094 | nieuwe|0.08971752251433238 | 
 | besluitvorming|1.2350000464525832 | illegalen|0.9519869291654123 | werken|0.5235918083110807 | raad|0.4384877053663824 | terwijl|0.4312929596078793 | parool|0.4331998091204545 | rond|0.3126644820491812 | ten|0.10346131894667421 | financiële|0.14995005694900296 | minister|0.08971752251433238 | 
 | kamerstuk|1.2350000464525832 | vrouwen|0.921515378344415 | februari|0.5235918083110807 | beleid|0.43832048389075523 | illegale|0.4312929596078793 | leerlingen|0.42182023608953495 | waaruit|0.3030920073427529 | ingezonden|0.09800707242626572 | gemeenten|0.14754357014418906 | gemeente|0.08719171964849304 | 
 | openbaar|1.1961803940344073 | eerder|0.9199242689056177 | geld|0.523276565105587 | zodat|0.43832048389075523 | dreigende|0.4312929596078793 | eigen|0.4174863083513096 | grond|0.3026072959636347 | nrc|0.09546600180671586 | grote|0.14754357014418906 | ingezonden|0.08515537469440489 | 
 | zaken|1.1961803940344073 | moeten|0.9199242689056177 | raad|0.523276565105587 | termijn|0.43832048389075523 | leerlingen|0.4312929596078793 | zodat|0.4174863083513096 | maken|0.30208611285225406 | telegraaf|0.09091152303913877 | waardoor|0.14221063321973113 | vragen|0.08291379429889796 | 
 | vanwege|1.1957253758032738 | bestrijdingsmiddelen|0.9199242689056177 | hiervan|0.5231518324974636 | rond|0.42507361646119357 | onderwerp|0.4176621551848674 | af|0.4174863083513096 | betuwelijn|0.30208611285225406 | augustus|0.08866780583006752 | college|0.14221063321973113 | nrc|0.08283387787291008 | 
 | programma|1.1957253758032738 | particuliere|0.9199242689056177 | heer|0.5231518324974636 | onderwerp|0.423594520911518 | rond|0.41688237343826895 | raad|0.4174863083513096 | sociale|0.30208611285225406 | s|0.08444112272332685 | bedrijven|0.13771109729435888 | berichten|0.08061483219359152 | 
 | vanuit|1.1957253758032738 | programma|0.9199242689056177 | vergaderjaar|0.5231518324974636 | gelderlander|0.423594520911518 | bestuur|0.4167470939145914 | toelating|0.4174863083513096 | courant|0.30208611285225406 | tussen|0.08038508208090653 | brief|0.13771109729435888 | europese|0.08029500580793292 | 
 | stellen|1.1954146978974496 | juni|0.9196963023883049 | af|0.5231518324974636 | twee|0.423594520911518 | drie|0.4167470939145914 | februari|0.4174863083513096 | gaat|0.29291472291494597 | algemeen|0.08038508208090653 | besluit|0.13750250552532572 | telegraaf|0.07819406862476924 | 
 | particuliere|1.1954146978974496 | regionale|0.9196963023883049 | amerikaanse|0.5082516366580789 | krijgen|0.423594520911518 | staat|0.4167470939145914 | terwijl|0.41730746259665547 | aanhangsel|0.2925951168765486 | dienen|0.08037501158894592 | terzake|0.13750250552532572 | augustus|0.07618003754202737 | 
 | metro|1.1954146978974496 | artikelen|0.9196963023883049 | aanzien|0.5070706172448467 | blijkt|0.423594520911518 | krijgen|0.4167470939145914 | opdracht|0.41730746259665547 | medewerkers|0.2925951168765486 | betreft|0.08037501158894592 | mogelijk|0.13366390521426835 | s|0.07238892555891441 | 
 | kamer|1.195299586004128 | mensen|0.9196963023883049 | commissie|0.5066580464800621 | commissie|0.423594520911518 | sociale|0.4063350247383267 | dreigende|0.41730746259665547 | tweede|0.2924539312231281 | feit|0.07855007332107916 | leden|0.13311691779670726 | algemeen|0.0690717495490364 | 
 | extra|1.195299586004128 | metro|0.9196963023883049 | onderwerp|0.5063837133466507 | betuwelijn|0.41483506136963477 | grond|0.403616337224318 | werken|0.41730746259665547 | name|0.2924539312231281 | aantal|0.07711940241624132 | onderwijs|0.13311691779670726 | tussen|0.06889582596519846 | 
 | eerder|1.195299586004128 | vanwege|0.9196963023883049 | asielzoekers|0.5063837133466507 | schiphol|0.4116236966338273 | maken|0.403616337224318 | scholen|0.40828139889363346 | televisieprogramma|0.2924539312231281 | waarin|0.07675019186778469 | problemen|0.1290147374809968 | betreft|0.06887986164064677 | 
 | bedrijf|1.195299586004128 | parool|0.9196963023883049 | nova|0.5062947561954741 | grond|0.4104005079486396 | medische|0.403616337224318 | gelderlander|0.4035934039792471 | gemeenten|0.2837314571105804 | vragen|0.07578987304229194 | gaan|0.12531476049515844 | feit|0.06722016545578935 | 
 | gezien|1.195299586004128 | extra|0.9196963023883049 | laten|0.5062947561954741 | courant|0.41030084992873905 | name|0.3919821085846754 | drie|0.40326486448692944 | grote|0.2837314571105804 | mogelijke|0.07186021720132008 | kinderen|0.12516354940022184 | juli|0.06603800428122543 | 
 | cobouw|1.195299586004128 | stellen|0.9196963023883049 | kosovo|0.49432268388470224 | kosten|0.398888741824345 | gaat|0.39172893876882686 | bestuur|0.40326486448692944 | amsterdam|0.28361627291962693 | nederland|0.06729488135137957 | waarbij|0.12153620115568654 | onderzoek|0.06420914192528114 | 
 | juni|1.195258564123053 | zaken|0.919665550008509 | maken|0.49113047116750774 | uitspraken|0.3982608929127706 | kosten|0.39172893876882686 | blijkt|0.40326486448692944 | aanvullende|0.28361627291962693 | aanvulling|0.06721795641711495 | ministerie|0.11810972212251547 | mogelijke|0.06123429425108053 | 
 | regio|1.195258564123053 | regio|0.919665550008509 | medische|0.491073303995748 | tweede|0.3982608929127706 | tweede|0.39172893876882686 | laten|0.40326486448692944 | waardoor|0.27620014820344224 | december|0.059302499137191426 | maatregelen|0.11810972212251547 | aanvulling|0.05711709769812872 | 
 | groningen|1.195258564123053 | koninklijke|0.919665550008509 | uitspraken|0.4777736230765979 | name|0.39818984001188573 | uitspraken|0.39172893876882686 | twee|0.4031217929529959 | handelingen|0.27554660056607105 | lid|0.058187567111619674 | trouw|0.114967924972547 | zie|0.0570609151476441 | 
 | moeten|1.195258564123053 | bedrijf|0.919665550008509 | hulp|0.47743773784084215 | aanhangsel|0.39818984001188573 | hulp|0.39165405318841984 | justitie|0.4031217929529959 | algemene|0.2754552509094157 | volkskrant|0.05702363854293068 | gevolgen|0.11249312588162058 | december|0.04994498593615419 | 
 | komen|1.195258564123053 | kamer|0.919665550008509 | informatie|0.47743773784084215 | televisieprogramma|0.39818984001188573 | grote|0.3812969736908367 | staat|0.4031217929529959 | besluit|0.2681879285825963 | nederlandse|0.05477131911313306 | jaar|0.1118727922002826 | lid|0.04877337427617975 | 
 | koninklijke|1.195258564123053 | relatie|0.919665550008509 | aanhangsel|0.47723097014379456 | informatie|0.39818984001188573 | aanvullende|0.3809463973426338 | nova|0.4031217929529959 | bedrijven|0.26787741472011156 | ter|0.05464123953150102 | inzake|0.11178368658787537 | volkskrant|0.0477231194693428 | 
 | parool|1.195258564123053 | herdruk|0.919665550008509 | medewerkers|0.47723097014379456 | gaat|0.39818984001188573 | gemeenten|0.3809463973426338 | krijgen|0.4031217929529959 | wet|0.26080743085611646 | betreffen|0.05304036129003156 | gemeente|0.10885264555035119 | ter|0.04574357127012523 | 
 | artikelen|1.195258564123053 | financiering|0.919665550008509 | name|0.47723097014379456 | aanvullende|0.38711697503714326 | financiële|0.3806729497082062 | aanzien|0.4031217929529959 | gebruik|0.2547289670670788 | eerdere|0.05046390301213964 | per|0.10339046440780862 | nederlandse|0.04574357127012523 | 
 | uitzetting|1.1642757294150667 | ziekenhuizen|0.8957260691054442 | gemeenten|0.4644655948348839 | grote|0.38711697503714326 | amsterdam|0.3806729497082062 | waaruit|0.3904281491174006 | problemen|0.2542135261807035 | dagblad|0.04965301482636661 | nrc|0.10339046440780862 | betreffen|0.04378781195427231 | 
 | illegale|1.1609787554729174 | voornemen|0.8916857893328463 | politie|0.4644655948348839 | amsterdam|0.38711697503714326 | algemene|0.3720090897337518 | courant|0.3903168663261114 | rapport|0.25418368264882035 | januari|0.04776672430741776 | europese|0.10225301227199417 | aanh|0.04299987299152687 | 
 | dreigende|1.1609787554729174 | hiervan|0.8913606637566786 | via|0.452694594536602 | gemeenten|0.38711697503714326 | college|0.37055249271301194 | tweede|0.37938587638172794 | waarbij|0.24207901845005933 | september|0.044855810488821576 | berichten|0.10083826679060962 | eerdere|0.04210560243656213 | 
 | opdracht|1.160595806734312 | toelating|0.8913606637566786 | handelingen|0.4526733149276666 | financiële|0.38707192494376774 | via|0.37052744565526974 | kosten|0.3787347134497664 | ministerie|0.23651917880858175 | oktober|0.0415630821783969 | telegraaf|0.0983932888367662 | hand|0.04210560243656213 | 
 | raad|1.1603425757985273 | eigen|0.8913606637566786 | brief|0.4417560200068754 | handelingen|0.3770156923871273 | waardoor|0.37052744565526974 | gaat|0.37865203553114973 | handelsblad|0.23133794767206745 | november|0.031215761829346683 | s|0.09162769823210815 | dagblad|0.041662129843505986 | 
 | geld|1.1603425757985273 | illegale|0.8913606637566786 | gevolg|0.4315423418850265 | via|0.37679975588771814 | bedrijven|0.361970427515442 | televisieprogramma|0.37865203553114973 | trouw|0.23124473929214232 | artikel|0.02212417277761855 | tussen|0.08753032739019882 | januari|0.03939365498315841 | 
 | auto|1.1603425757985273 | werken|0.8911906408760785 | mogelijk|0.4315423418850265 | algemene|0.37679975588771814 | besluit|0.36127990648477015 | medewerkers|0.37865203553114973 | inzake|0.22629296766078016 | aanleiding|0.01338637196978161 | betreft|0.08750561133047857 | september|0.03698367783800239 | 
 | zodat|1.1603425757985273 | termijn|0.8911906408760785 | leden|0.4314786234393677 | terzake|0.3677187459823048 | brief|0.36127990648477015 | amsterdam|0.3682336037202584 | nieuwe|0.22622913958896482 | bericht|0.00626152998780859 | dienen|0.08750561133047857 | oktober|0.03405258210923096 | 


## 6.
Evaluate your classifiers using Precision, Recall and F1. (Give a table in which you show these values for using the top 10, top 100 terms and all terms, for all of your 10 classes)
          Thus do feature selection per class, and use for each class the top n best features for that class. 
          <br/>
      Also show the microaverage(s) for all 10 classes together.
      <br/>
      If you like you can also present this in a figure like MRS.13.8. 
      Then compute the F1 measure for the same number of terms as in that figure.

In [ ]:


def precision():
    raise NotImplemented
    
def recall():
    raise NotImplemented
    
def F1_measure():
    P = precision()
    R = recall()
    return 2*P*R/(P+R)

In [ ]:
def getstats()
    ApplyMultinomialNB(classes, priors, conditionals, d, Vocab=None)

## 7 
You have done the complete implementation by yourself. Congratulations! You can also use `scikit-learn` routines for all of this work. Do that. So follow [this text classification tutorial](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)  and implement the same steps but now with your kamervragen dataset. Also use [mutual information feature selection](http://scikit-learn.org/stable/modules/feature_selection.html) to select the K-best features, and compare the results as before.


## 8

Reflect and report briefly about your choices in this process and about the obtained results. Also reflect on the differences between the scikit learn approach and the "own implementation approach".

<h3>Training/Testing</h3>
<p>It is important that you do not test your classifier using documents that have also been used in training.
    So split up your collection in a training set and a test set. A 80%-20% split is reasonable.

<br/>
    If you have too little data you can use 5 or <a href="http://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation">10-fold cross validation</a>.</p>



<h2>Form of handing in your final product</h2>

* An IPython notebook with for each question, a MarkDown cell containing the question, a code cell which solves the question, an output cell with the output, followed by a MarkDown cell with explanation/reflection  